


# **Clasificación de imágenes con métodos clásicos y redes neuronales**



*   Raúl Alejandro Pérez Saucedo A01566937
*   Laura Merarí Valdivia Frausto A01641790
*   Francisco Javier Chávez Ochoa A01641644
*   Mayra Sarahí de Luna Castillo A01635774

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, recall_score
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from sklearn.neural_network import MLPClassifier

from skimage import io
from skimage.transform import resize
from skimage.feature import graycomatrix, graycoprops
from skimage.color import rgb2gray
from skimage import img_as_ubyte
from sklearn.model_selection import cross_val_predict, StratifiedKFold

import pickle
from os import listdir
from os.path import isfile, join

In [ ]:
from keras.datasets import fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Fashion-MNIST

### Modelo lineal

In [ ]:
x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_test_flat = x_test.reshape(x_test.shape[0], -1)

linear_svm = SVC(kernel='linear')
linear_svm.fit(x_train_flat, y_train)

y_pred_svm_linear = linear_svm.predict(x_test_flat)

accuracy_linear = accuracy_score(y_test, y_pred_svm_linear)
recall_per_class = recall_score(y_test, y_pred_svm_linear, average=None)

print("Accuracy del modelo lineal SVM:", accuracy_linear)
for class_idx, recall in enumerate(recall_per_class):
    print(f"Recall para la clase {class_idx}: {recall}")

Accuracy del modelo lineal SVM: 0.8463
Recall para la clase 0: 0.815
Recall para la clase 1: 0.962
Recall para la clase 2: 0.769
Recall para la clase 3: 0.842
Recall para la clase 4: 0.773
Recall para la clase 5: 0.936
Recall para la clase 6: 0.562
Recall para la clase 7: 0.934
Recall para la clase 8: 0.925
Recall para la clase 9: 0.945


### Modelo no lineal

In [ ]:
rbf_svm = SVC(kernel='rbf')
rbf_svm.fit(x_train_flat, y_train)

y_pred_svm_rbf = rbf_svm.predict(x_test_flat)

accuracy_rbf = accuracy_score(y_test, y_pred_svm_rbf)
recall_per_class = recall_score(y_test, y_pred_svm_rbf, average=None)

print("Accuracy del modelo no lineal:", accuracy_rbf)
for class_idx, recall in enumerate(recall_per_class):
    print(f"Recall para la clase {class_idx}: {recall}")

Accuracy del modelo no lineal: 0.8829
Recall para la clase 0: 0.857
Recall para la clase 1: 0.962
Recall para la clase 2: 0.816
Recall para la clase 3: 0.89
Recall para la clase 4: 0.815
Recall para la clase 5: 0.951
Recall para la clase 6: 0.655
Recall para la clase 7: 0.955
Recall para la clase 8: 0.977
Recall para la clase 9: 0.951


### Modelo perceptrón multicapa

In [ ]:
hidden_layer_sizes_options = [(50,), (100,), (50, 50), (100, 50), (100, 100)]

best_accuracy = 0
best_config = None

for hidden_layer_config in hidden_layer_sizes_options:
    mlp = MLPClassifier(hidden_layer_sizes=hidden_layer_config, max_iter=200, random_state=42)
    mlp.fit(x_train_flat, y_train)

    y_pred_test = mlp.predict(x_test_flat)
    accuracy = accuracy_score(y_test, y_pred_test)

    print(f"Configuración {hidden_layer_config}: Accuracy en test = {accuracy}")

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_config = hidden_layer_config

final_mlp = MLPClassifier(hidden_layer_sizes=best_config, max_iter=200, random_state=42)
final_mlp.fit(x_train_flat, y_train)

y_pred_test_final = final_mlp.predict(x_test_flat)
accuracy_final = accuracy_score(y_test, y_pred_test_final)
recall_final = recall_score(y_test, y_pred_test_final, average=None)

print("Mejor configuración de capas y neuronas:", best_config)
print("Accuracy:", accuracy_final)
for class_idx, recall in enumerate(recall_final):
    print(f"Recall para la clase {class_idx}: {recall}")

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Configuración (50,): Accuracy en test = 0.8719


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Configuración (100,): Accuracy en test = 0.8848


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Configuración (50, 50): Accuracy en test = 0.8733
Configuración (100, 50): Accuracy en test = 0.8834
Configuración (100, 100): Accuracy en test = 0.8839
Mejor configuración de capas y neuronas: (100,)
Accuracy: 0.8848
Recall para la clase 0: 0.835
Recall para la clase 1: 0.968
Recall para la clase 2: 0.842
Recall para la clase 3: 0.89
Recall para la clase 4: 0.818
Recall para la clase 5: 0.951
Recall para la clase 6: 0.666
Recall para la clase 7: 0.967
Recall para la clase 8: 0.96
Recall para la clase 9: 0.951


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


# Fotografías satelitales

In [ ]:
#------------------------------------------------------------------------------------------------------------------
#Load image files
#------------------------------------------------------------------------------------------------------------------
scale = 8
img_width = int(1920/scale)
img_height = int(1080/scale)

paths = ['/content/drive/MyDrive/Semestre 5/Redes Neuronales/Biomas/Agua/', '/content/drive/MyDrive/Semestre 5/Redes Neuronales/Biomas/Bosque/', '/content/drive/MyDrive/Semestre 5/Redes Neuronales/Biomas/Ciudad/', '/content/drive/MyDrive/Semestre 5/Redes Neuronales/Biomas/Cultivo/', '/content/drive/MyDrive/Semestre 5/Redes Neuronales/Biomas/Desierto/', '/content/drive/MyDrive/Semestre 5/Redes Neuronales/Biomas/Montaña/']

images = []
labels = []
for label, p in enumerate(paths):
    files = [f for f in listdir(p) if isfile(join(p, f))]
    for f in files:
        print("Loading:", p + f)
        labels.append(label)
        rgb = io.imread(p + f)
        rgb_resized = resize(rgb, (img_height, img_width), anti_aliasing=True)
        images.append(rgb_resized)


n_img = len(images)
labels = np.array(labels)

In [ ]:
#------------------------------------------------------------------------------------------------------------------
#   Color histograms
#------------------------------------------------------------------------------------------------------------------
nbins = 16
hist_descriptor = []
for rgb_resized in images:
  rh = np.histogram(rgb_resized[:,:,0].flatten(), nbins, density = True)
  gh = np.histogram(rgb_resized[:,:,1].flatten(), nbins, density = True)
  bh = np.histogram(rgb_resized[:,:,2].flatten(), nbins, density = True)
  hist_descriptor.append(np.concatenate((rh[0], gh[0], bh[0])))

#------------------------------------------------------------------------------------------------------------------
#   Texture descriptors
#------------------------------------------------------------------------------------------------------------------
texture_desc = []

for rgb_resized in images:
  gray_resized = img_as_ubyte(rgb2gray(rgb_resized))
  glcm = graycomatrix(gray_resized, distances=[5], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4])

  texture_desc.append([graycoprops(glcm, 'dissimilarity')[0, 0], graycoprops(glcm, 'homogeneity')[0, 0], graycoprops(glcm, 'energy')[0, 0], graycoprops(glcm, 'correlation')[0, 0]])

### Modelo lineal

In [ ]:
x = np.concatenate((texture_desc, hist_descriptor), axis=1)

linear_svm = SVC(kernel='linear')

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

y_pred_svm_linear = cross_val_predict(linear_svm, x, labels, cv=cv)

recall_per_class = recall_score(labels, y_pred_svm_linear, average=None)

accuracy = accuracy_score(labels, y_pred_svm_linear)

for class_idx, recall in enumerate(recall_per_class):
    print(f"Recall para la clase {class_idx}: {recall}")

print("Accuracy:", accuracy)

Recall para la clase 0: 0.8656716417910447
Recall para la clase 1: 0.8444444444444444
Recall para la clase 2: 0.9375
Recall para la clase 3: 0.7238805970149254
Recall para la clase 4: 0.9011976047904192
Recall para la clase 5: 0.8391812865497076
Accuracy: 0.8567639257294429


### Modelo no lineal

In [ ]:
linear_svm = SVC(kernel='rbf')

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

y_pred_svm_linear = cross_val_predict(linear_svm, x, labels, cv=cv)

recall_per_class = recall_score(labels, y_pred_svm_linear, average=None)

accuracy = accuracy_score(labels, y_pred_svm_linear)

for class_idx, recall in enumerate(recall_per_class):
    print(f"Recall para la clase {class_idx}: {recall}")

print("Accuracy:", accuracy)

Recall para la clase 0: 0.826865671641791
Recall para la clase 1: 0.8185185185185185
Recall para la clase 2: 0.9047619047619048
Recall para la clase 3: 0.6119402985074627
Recall para la clase 4: 0.9341317365269461
Recall para la clase 5: 0.847953216374269
Accuracy: 0.8318302387267904


### Modelo Perceptron

In [ ]:
hidden_layer_sizes_options = [(50,), (100,), (50, 50), (100, 50), (100, 100)]

best_accuracy = 0
best_config = None

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for hidden_layer_config in hidden_layer_sizes_options:
    mlp = MLPClassifier(hidden_layer_sizes=hidden_layer_config, max_iter=200, random_state=42)

    y_pred_cv = cross_val_predict(mlp, x, labels, cv=cv)

    accuracy = accuracy_score(labels, y_pred_cv)

    print(f"Configuración {hidden_layer_config}: Accuracy promedio en validación cruzada = {accuracy}")

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_config = hidden_layer_config

final_mlp = MLPClassifier(hidden_layer_sizes=best_config, max_iter=200, random_state=42)
final_mlp.fit(x, labels)

y_pred_final = final_mlp.predict(x)

recall_final = recall_score(labels, y_pred_final, average=None)

print("Mejor configuración de capas y neuronas:", best_config)
print("Accuracy promedio en validación cruzada:", best_accuracy)
for class_idx, recall in enumerate(recall_final):
    print(f"Recall para la clase {class_idx}: {recall}")

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptro

Configuración (50,): Accuracy promedio en validación cruzada = 0.8742705570291777


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptro

Configuración (100,): Accuracy promedio en validación cruzada = 0.8748010610079575


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptro

...

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptro

Configuración (100, 50): Accuracy promedio en validación cruzada = 0.8774535809018568


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptro

Configuración (100, 100): Accuracy promedio en validación cruzada = 0.8769230769230769
Mejor configuración de capas y neuronas: (100, 50)
Accuracy promedio en validación cruzada: 0.8774535809018568
Recall para la clase 0: 1.0
Recall para la clase 1: 0.9962962962962963
Recall para la clase 2: 0.9970238095238095
Recall para la clase 3: 0.996268656716418
Recall para la clase 4: 0.9970059880239521
Recall para la clase 5: 0.9970760233918129


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


# Imágenes de objetos

In [ ]:
#------------------------------------------------------------------------------------------------------------------
#Load image files
#------------------------------------------------------------------------------------------------------------------
scale = 8
img_width = int(1920/scale)
img_height = int(1080/scale)

paths = ['/content/drive/MyDrive/Semestre 5/Redes Neuronales/Imágenes SP/Celular/',
         '/content/drive/MyDrive/Semestre 5/Redes Neuronales/Imágenes SP/Hats/',
         '/content/drive/MyDrive/Semestre 5/Redes Neuronales/Imágenes SP/Lapiz/',
         '/content/drive/MyDrive/Semestre 5/Redes Neuronales/Imágenes SP/Libro/',
         '/content/drive/MyDrive/Semestre 5/Redes Neuronales/Imágenes SP/Shoes/']

images = []
labels = []
for label, p in enumerate(paths):
    files = [f for f in listdir(p) if isfile(join(p, f))]
    for f in files:
        print("Loading:", p + f)
        labels.append(label)
        rgb = io.imread(p + f)
        rgb_resized = resize(rgb, (img_height, img_width), anti_aliasing=True)
        images.append(rgb_resized)


n_img = len(images)
labels = np.array(labels)

#------------------------------------------------------------------------------------------------------------------
#   Color histograms
#------------------------------------------------------------------------------------------------------------------
nbins = 16
hist_descriptor = []
for rgb_resized in images:
  rh = np.histogram(rgb_resized[:,:,0].flatten(), nbins, density = True)
  gh = np.histogram(rgb_resized[:,:,1].flatten(), nbins, density = True)
  bh = np.histogram(rgb_resized[:,:,2].flatten(), nbins, density = True)
  hist_descriptor.append(np.concatenate((rh[0], gh[0], bh[0])))

#------------------------------------------------------------------------------------------------------------------
#   Texture descriptors
#------------------------------------------------------------------------------------------------------------------
texture_desc = []

for rgb_resized in images:
  gray_resized = img_as_ubyte(rgb2gray(rgb_resized))
  glcm = graycomatrix(gray_resized, distances=[5], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4])

  texture_desc.append([graycoprops(glcm, 'dissimilarity')[0, 0], graycoprops(glcm, 'homogeneity')[0, 0], graycoprops(glcm, 'energy')[0, 0], graycoprops(glcm, 'correlation')[0, 0]])

### Modelo lineal

In [ ]:
x = np.concatenate((texture_desc, hist_descriptor), axis=1)

linear_svm = SVC(kernel='linear')

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

y_pred_svm_linear = cross_val_predict(linear_svm, x, labels, cv=cv)

recall_per_class = recall_score(labels, y_pred_svm_linear, average=None)

accuracy = accuracy_score(labels, y_pred_svm_linear)

for class_idx, recall in enumerate(recall_per_class):
    print(f"Recall para la clase {class_idx}: {recall}")

print("Accuracy:", accuracy)


Recall para la clase 0: 0.984
Recall para la clase 1: 0.994
Recall para la clase 2: 1.0
Recall para la clase 3: 1.0
Recall para la clase 4: 0.998
Accuracy: 0.9952


### Modelo no lineal

In [ ]:
linear_svm = SVC(kernel='rbf')

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

y_pred_svm_linear = cross_val_predict(linear_svm, x, labels, cv=cv)

recall_per_class = recall_score(labels, y_pred_svm_linear, average=None)

accuracy = accuracy_score(labels, y_pred_svm_linear)

for class_idx, recall in enumerate(recall_per_class):
    print(f"Recall para la clase {class_idx}: {recall}")

print("Accuracy:", accuracy)

Recall para la clase 0: 0.958
Recall para la clase 1: 0.982
Recall para la clase 2: 1.0
Recall para la clase 3: 1.0
Recall para la clase 4: 0.998
Accuracy: 0.9876


### Modelo Percpetron

In [ ]:
hidden_layer_sizes_options = [(50,), (100,), (50, 50), (100, 50), (100, 100)]

best_accuracy = 0
best_config = None

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for hidden_layer_config in hidden_layer_sizes_options:
    mlp = MLPClassifier(hidden_layer_sizes=hidden_layer_config, max_iter=200, random_state=42)

    y_pred_cv = cross_val_predict(mlp, x, labels, cv=cv)

    accuracy = accuracy_score(labels, y_pred_cv)

    print(f"Configuración {hidden_layer_config}: Accuracy promedio en validación cruzada = {accuracy}")

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_config = hidden_layer_config

final_mlp = MLPClassifier(hidden_layer_sizes=best_config, max_iter=200, random_state=42)
final_mlp.fit(x, labels)

y_pred_final = final_mlp.predict(x)

recall_final = recall_score(labels, y_pred_final, average=None)

print("Mejor configuración de capas y neuronas:", best_config)
print("Accuracy promedio en validación cruzada:", best_accuracy)
for class_idx, recall in enumerate(recall_final):
    print(f"Recall para la clase {class_idx}: {recall}")

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Configuración (50,): Accuracy promedio en validación cruzada = 0.9964
Configuración (100,): Accuracy promedio en validación cruzada = 0.9988
Configuración (50, 50): Accuracy promedio en validación cruzada = 0.9988
Configuración (100, 50): Accuracy promedio en validación cruzada = 0.9976
Configuración (100, 100): Accuracy promedio en validación cruzada = 0.998
Mejor configuración de capas y neuronas: (100,)
Accuracy promedio en validación cruzada: 0.9988
Recall para la clase 0: 1.0
Recall para la clase 1: 1.0
Recall para la clase 2: 1.0
Recall para la clase 3: 1.0
Recall para la clase 4: 1.0
